In [47]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
from keras_contrib.layers import CRF


epochs = 50
trainable=True

In [48]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [49]:
def get_sentencs(path):
    sentences=[]
    sentence=[]
    with open(path,'rb') as f:
        for line in f:
            splits = line.split()
            try:
                word = splits[0].decode()
                label = splits[1].decode()
                sentence.append([word, label])
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences

In [50]:
trainSentences = get_sentencs('/srv/Resources/CONLL2002/esp.train')
testSentences = get_sentencs('/srv/Resources/CONLL2002/esp.testa')
devSentences = get_sentencs('/srv/Resources/CONLL2002/esp.testb')

In [51]:
print(trainSentences[0])

[['Melbourne', 'B-LOC'], ['(', 'O'], ['Australia', 'B-LOC'], [')', 'O'], [',', 'O'], ['25', 'O'], ['may', 'O'], ['(', 'O'], ['EFE', 'B-ORG'], [')', 'O'], ['.', 'O']]


In [52]:
trainSentences = addCharInformatioin(trainSentences)
devSentences = addCharInformatioin(devSentences)
testSentences = addCharInformatioin(testSentences)


In [53]:
print(trainSentences[0])

[['Melbourne', ['M', 'e', 'l', 'b', 'o', 'u', 'r', 'n', 'e'], 'B-LOC'], ['(', ['('], 'O'], ['Australia', ['A', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a'], 'B-LOC'], [')', [')'], 'O'], [',', [','], 'O'], ['25', ['2', '5'], 'O'], ['may', ['m', 'a', 'y'], 'O'], ['(', ['('], 'O'], ['EFE', ['E', 'F', 'E'], 'B-ORG'], [')', [')'], 'O'], ['.', ['.'], 'O']]


In [54]:
labelSet = set()
words = {}
characters=set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
            words[token.lower()] = True

In [55]:
print(labelSet)

{'O', 'I-ORG', 'I-LOC', 'B-PER', 'B-LOC', 'B-ORG', 'I-MISC', 'I-PER', 'B-MISC'}


In [56]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [57]:
print(label2Idx)

{'I-PER': 7, 'B-ORG': 5, 'B-PER': 3, 'I-ORG': 1, 'I-MISC': 6, 'B-MISC': 8, 'B-LOC': 4, 'O': 0, 'I-LOC': 2}


In [58]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [59]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/srv/fastText/spanish_words.txt','wb')
f.write(string_words.encode())
f.close()

In [69]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('/srv/jindal/NER/spanish_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [70]:
print(len(wordEmbeddings[2]))

300


In [71]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'f': 42, ':': 0, '5': 1, 'm': 3, 'J': 49, 'W': 4, '8': 5, 'n': 51, '?': 52, 'X': 53, 'P': 6, 'o': 7, 'b': 54, 'C': 55, 'y': 77, 'F': 9, '-': 57, 'L': 10, 'x': 11, '(': 12, '!': 13, 'Q': 14, 'ü': 56, 'w': 59, 'd': 15, 'v': 16, '·': 61, ';': 62, '%': 50, 'K': 64, 'V': 17, 'a': 65, 'à': 66, 'N': 67, 'z': 18, '&': 90, 'B': 19, 'ñ': 68, '/': 20, 'Z': 2, 'p': 73, '2': 22, 'Ñ': 23, 'i': 8, 'j': 69, '=': 70, '3': 71, '9': 24, 'ú': 25, 'u': 72, '0': 74, 'e': 26, 'G': 27, 'D': 88, '7': 75, 's': 82, 'O': 76, '¿': 58, 'c': 28, '@': 78, '6': 79, '1': 29, 'S': 30, 'H': 80, 'á': 81, 'k': 31, '"': 32, '¡': 83, '+': 33, "'": 34, 'U': 35, ',': 36, '4': 37, 'é': 84, 't': 38, 'E': 39, ')': 40, 'T': 41, 'q': 44, 'l': 43, 'I': 63, 'R': 45, '.': 85, 'Y': 86, 'A': 87, 'h': 46, 'r': 60, 'í': 47, 'M': 89, 'g': 48, 'ó': 21}


In [72]:
print(trainSentences[0]) 

[['Melbourne', ['M', 'e', 'l', 'b', 'o', 'u', 'r', 'n', 'e'], 'B-LOC'], ['(', ['('], 'O'], ['Australia', ['A', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a'], 'B-LOC'], [')', [')'], 'O'], [',', [','], 'O'], ['25', ['2', '5'], 'O'], ['may', ['m', 'a', 'y'], 'O'], ['(', ['('], 'O'], ['EFE', ['E', 'F', 'E'], 'B-ORG'], [')', [')'], 'O'], ['.', ['.'], 'O']]


In [73]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [74]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
# print((train_set[0][2]))

[[12262, 8498, 22291, 15973, 20006, 7908, 10883, 8498, 27495, 15973, 23662], [3, 4, 3, 4, 4, 0, 1, 4, 2, 4, 4], array([[89, 26, 43, 54,  7, 72, 60, 51, 26,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [87, 72, 82, 38, 60, 65, 43,  8, 65,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [40,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [75]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [76]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2730        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_10 (TimeDistri (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [77]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch
        labels = labels.tolist()
#         print(labels)
        for sentence in labels:
            for i,label in enumerate(sentence):
                temp = [0]*len(label2Idx)
                value = label[0]
                temp[value]=1
                sentence[i] = temp
        labels = np.array(labels)
#         print(labels)
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
#         print('*')
        print(' ')

Epoch 0/50
  0/109 [..............................] - ETA: 0s 
 
  2/109 [..............................] - ETA: 5:25 
 
  4/109 [>.............................] - ETA: 2:42 
 
111/109 [==============================] - 34s 303ms/step
 
112/109 [==============================] - 34s 302ms/step
 
113/109 [===============================] - 34s 302ms/step
 
120/109 [=================================] - 34s 285ms/step
 
123/109 [=================================] - 34s 280ms/step
 
137/109 [=====================================] - 35s 253ms/step
 
139/109 [======================================] - 35s 251ms/step
 
142/109 [=======================================] - 35s 248ms/step
 
177/109 [================================================] - 35s 201ms/step
 
1237/109 [================================================================================================================================================================================================================================

111/109 [==============================] - 27s 245ms/step
 
112/109 [==============================] - 27s 245ms/step
 
113/109 [===============================] - 28s 245ms/step
 
120/109 [=================================] - 28s 232ms/step
 
123/109 [=================================] - 28s 227ms/step
 
137/109 [=====================================] - 28s 206ms/step
 
139/109 [======================================] - 28s 205ms/step
 
142/109 [=======================================] - 29s 203ms/step
 
177/109 [================================================] - 29s 164ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 31s 25ms/step
 
260/109 [================================

111/109 [==============================] - 28s 254ms/step
 
112/109 [==============================] - 28s 253ms/step
 
113/109 [===============================] - 28s 252ms/step
 
120/109 [=================================] - 29s 240ms/step
 
123/109 [=================================] - 29s 236ms/step
 
137/109 [=====================================] - 29s 214ms/step
 
139/109 [======================================] - 30s 213ms/step
 
142/109 [=======================================] - 30s 210ms/step
 
177/109 [================================================] - 30s 170ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 32s 26ms/step
 
260/109 [================================

 
123/109 [=================================] - 28s 230ms/step
 
137/109 [=====================================] - 29s 208ms/step
 
139/109 [======================================] - 29s 207ms/step
 
142/109 [=======================================] - 29s 204ms/step
 
177/109 [================================================] - 29s 166ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 31s 25ms/step
 
260/109 [=======================================================================] - 32s 122ms/step                                                                                                                                                                                         

111/109 [==============================] - 28s 249ms/step
 
112/109 [==============================] - 28s 249ms/step
 
113/109 [===============================] - 28s 248ms/step
 
120/109 [=================================] - 28s 235ms/step
 
123/109 [=================================] - 28s 231ms/step
 
137/109 [=====================================] - 29s 209ms/step
 
139/109 [======================================] - 29s 208ms/step
 
142/109 [=======================================] - 29s 205ms/step
 
177/109 [================================================] - 30s 167ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 32s 26ms/step
 
260/109 [================================

...

 
123/109 [=================================] - 26s 213ms/step
 
137/109 [=====================================] - 26s 193ms/step
 
139/109 [======================================] - 27s 192ms/step
 
142/109 [=======================================] - 27s 190ms/step
 
177/109 [================================================] - 27s 155ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 30s 24ms/step
 
260/109 [=======================================================================] - 30s 117ms/step                                                                                                                                                                                         

111/109 [==============================] - 25s 224ms/step
 
112/109 [==============================] - 25s 225ms/step
 
113/109 [===============================] - 25s 225ms/step
 
120/109 [=================================] - 26s 213ms/step
 
123/109 [=================================] - 26s 210ms/step
 
137/109 [=====================================] - 26s 190ms/step
 
139/109 [======================================] - 26s 190ms/step
 
142/109 [=======================================] - 27s 188ms/step
 
177/109 [================================================] - 27s 153ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 30s 24ms/step
 
260/109 [================================

111/109 [==============================] - 25s 226ms/step
 
112/109 [==============================] - 25s 226ms/step
 
113/109 [===============================] - 26s 226ms/step
 
120/109 [=================================] - 26s 215ms/step
 
123/109 [=================================] - 26s 212ms/step
 
137/109 [=====================================] - 26s 192ms/step
 
139/109 [======================================] - 27s 192ms/step
 
142/109 [=======================================] - 27s 190ms/step
 
177/109 [================================================] - 27s 154ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 30s 24ms/step
 
260/109 [================================

111/109 [==============================] - 25s 227ms/step
 
112/109 [==============================] - 25s 227ms/step
 
113/109 [===============================] - 26s 227ms/step
 
120/109 [=================================] - 26s 216ms/step
 
123/109 [=================================] - 26s 213ms/step
 
137/109 [=====================================] - 26s 193ms/step
 
139/109 [======================================] - 27s 193ms/step
 
142/109 [=======================================] - 27s 191ms/step
 
177/109 [================================================] - 27s 155ms/step
 
1237/109 [====================================================================================================================================================================================================================================================================================================================================================] - 30s 24ms/step
 
260/109 [================================

 
313/109 [======================================================================================] - 31s 100ms/step
 


In [78]:
predLabels, correctLabels = tag_dataset(dev_batch)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

1515/1516 [============================>.] - ETA: 0sDev-Data: Prec: 0.799, Rec: 0.821, F1: 0.810


In [79]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

1913/1914 [============================>.] - ETA: 0sTest-Data: Prec: 0.794, Rec: 0.816, F1: 0.805
